# Pandas 分层索引

Pandas 提供的 `Series` 与 `DataFrame` 对象用于有序、有映射的数据集。除了数据之外，还有行列索引（标签）。在前面几个章节中，已经介绍了 Pandas 中常用的索引类型：
- `RangeIndex`，
- `Int64Index`，整数索引
- `Float64Index`，浮点数索引
- `Index`， 通用索引类型
- `DatetimeIndex`, 日期型索引

分层索引是 Pandas 的重要特性，分层索引是指在一个轴上拥有多个索引层级，使得能够以低纬度形式来处理高维度数据。分层索引也被称为多层索引或层次化索引。本节介绍分层索引的定义以及常用操作方法。

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 分层索引定义

Pandas 的 `Series` 与 `DataFrame` 对象都可以使用分层索引。下面分别予以介绍。

### `Series`对象的分层索引

使用嵌套列表作为索引来创建`Series`对象时，就会定义一个分层索引：

In [10]:
idx_list = [
    ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd', 'd', 'd'],
    [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
]    
ser = pd.Series(np.random.randn(12), index=idx_list)
ser

a  0   -1.353864
   1    0.881489
   2   -1.764737
b  0    0.738785
   1    0.667537
   2   -1.689044
c  0    0.758787
   1    0.167569
   2   -1.462910
d  0   -0.699724
   1    0.472923
   2   -0.613440
dtype: float64

检查其索引所属类型，并打印索引内容：

In [16]:
# 索引对象的类
print(type(ser.index))
# 索引内容
ser.index

<class 'pandas.core.indexes.multi.MultiIndex'>


MultiIndex(levels=[['a', 'b', 'c', 'd'], [0, 1, 2]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])

可以看出，分层索引是`MultiIndex`的实例对象。索引对象的`values`实际上一个元组：

In [17]:
ser.index.values

array([('a', 0), ('a', 1), ('a', 2), ('b', 0), ('b', 1), ('b', 2),
       ('c', 0), ('c', 1), ('c', 2), ('d', 0), ('d', 1), ('d', 2)],
      dtype=object)

在分层索引对象中居左的索引常称为外层，居右的则称为内层。创建拥有分层索引的`Series`对象后，可以使用索引来快捷选取数据的子集：

In [18]:
# 外层选取
ser['c']

0    0.758787
1    0.167569
2   -1.462910
dtype: float64

In [19]:
# 选取多个
ser['a': 'b']

a  0   -1.353864
   1    0.881489
   2   -1.764737
b  0    0.738785
   1    0.667537
   2   -1.689044
dtype: float64

In [21]:
# 内层选取
ser[:, 2]

a   -1.764737
b   -1.689044
c   -1.462910
d   -0.613440
dtype: float64

### `DataFrame`对象的分层索引

`DataFrame`对象包括行与列索引，在这两个轴上都可以拥有分层索引。同样在创建`DataFrame`对象时，使用嵌套列表即可：

In [25]:
idx_list = [['Python', 'Python', 'C', 'C'], 
            [1, 2, 1, 2]]
column_list = [['小儿', '小儿', '成年'],
               ['G10', 'G14', 'G22']]
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                  index=idx_list, columns=column_list)
df

小儿      成年
         G10 G14 G22
Python 1   0   1   2
       2   3   4   5
C      1   6   7   8
       2   9  10  11

`DataFrame`对象的行与列索引对象都是`MultiIndex`的实例对象：

In [27]:
type(df.index), type(df.columns)

(pandas.core.indexes.multi.MultiIndex, pandas.core.indexes.multi.MultiIndex)

可以为分层索引对象的层级指定名字：

In [30]:
df.index.names = ['Langs', 'Level']
df.columns.names = ['年龄段', 'Age']
df

年龄段           小儿      成年
Age          G10 G14 G22
Langs  Level            
Python 1       0   1   2
       2       3   4   5
C      1       6   7   8
       2       9  10  11

同样，使用部分索引，可以快捷选取数据：

In [33]:
df['小儿']

Age           G10  G14
Langs  Level          
Python 1        0    1
       2        3    4
C      1        6    7
       2        9   10

分层索引对象可以使用`MultiIndex`类的`from_arrays()`来创建：

In [40]:
pd.MultiIndex.from_arrays(column_list, names=['年龄段', 'Age'])

MultiIndex(levels=[['小儿', '成年'], ['G10', 'G14', 'G22']],
           labels=[[0, 0, 1], [0, 1, 2]],
           names=['年龄段', 'Age'])

### 设置列为索引

`DataFrame`对象的方法`set_index()`可以把已有列设置为索引，使用多个列会创建多层索引，其使用语法为：
```
df.set_index(keys, drop=True, append=False, inplace=False, verify_integrity=False)
```
- 输入
    - `kyes`，列名
    - `drop=True`，是否删除设置为索引的列
    - `append=False`，是否保留原有索引
    - `inplace=False`，是否直接对原始对象进行修改
- 输出    
    - 返回一个新的`DataFrame`对象

In [45]:
df = pd.DataFrame({
    'langs': ['Python', 'Python', 'Python', 'C', 'C', 'C'],
    'level': [1, 2, 3, 1, 2, 3],
    'G10': np.arange(6),
    'G14': np.arange(6),
    'G22': np.arange(6),
})
df

,langs,level,G10,G14,G22
0,Python,1,0,0,0
1,Python,2,1,1,1
2,Python,3,2,2,2
3,C,1,3,3,3
4,C,2,4,4,4
5,C,3,5,5,5


In [53]:
# 把`langs, level`设置为索引：
newdf = df.set_index(['langs', 'level'])
newdf

G10  G14  G22
langs  level               
Python 1        0    0    0
       2        1    1    1
       3        2    2    2
C      1        3    3    3
       2        4    4    4
       3        5    5    5

In [51]:
# 不删除设置为索引的列：
df.set_index(['langs', 'level'], drop=False)

langs  level  G10  G14  G22
langs  level                              
Python 1      Python      1    0    0    0
       2      Python      2    1    1    1
       3      Python      3    2    2    2
C      1           C      1    3    3    3
       2           C      2    4    4    4
       3           C      3    5    5    5

使用`DataFrame`对象的方法`reset_index()`进行反操作，即把索引设置为列，其使用语法为：
```
df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
```
- 输入
    - `level=None`，整数、字符串、元组或列表。指定索引。缺省是全部索引。
    - `drop=False`，试着删除索引不插入到列中
- 输出    
    - 返回一个新的`DataFrame`对象

In [60]:
# 把索引转换为列
newdf.reset_index()

,langs,level,G10,G14,G22
0,Python,1,0,0,0
1,Python,2,1,1,1
2,Python,3,2,2,2
3,C,1,3,3,3
4,C,2,4,4,4
5,C,3,5,5,5


In [61]:
# 删除`level`索引
newdf.reset_index(level=['level'], drop=True)

,G10,G14,G22
langs,,,
Python,0,0,0
Python,1,1,1
Python,2,2,2
C,3,3,3
C,4,4,4
C,5,5,5


## 分层索引常用操作方法

对于分层索引常用的操作方法有：
- 交换分层
- 层级排序
- 选取数据

### 交换分层

`DataFrame`对象的方法`swaplevel()`来交换分层，只需传入层级序号或层级名称即可，该方法会返回一个新对象：

In [66]:
# 交换`langs`与`level`
newdf.swaplevel('langs', 'level')

,,G10,G14,G22
level,langs,,,
1,Python,0,0,0
2,Python,1,1,1
3,Python,2,2,2
1,C,3,3,3
2,C,4,4,4
3,C,5,5,5


### 层级排序

`DataFrame`对象的方法`sort_index()`来交换分层，其使用语法为：
```python
df.sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True, by=None)
```
主要参数
- `axis`，沿行或列进行排序
- `level`，指定索引层级
- `ascending=True`，升序或降序

该方法会返回新对象：

In [70]:
# 根据`level`索引来进行排序
newdf.sort_index(level='level')

,,G10,G14,G22
langs,level,,,
C,1,3,3,3
Python,1,0,0,0
C,2,4,4,4
Python,2,1,1,1
C,3,5,5,5
Python,3,2,2,2


### 选取数据

对于分层索引对象，使用`loc`运算符来选取数据时，行标签需要使用嵌套列表的元组来选取：

In [72]:
newdf.loc[(['Python', 'C'], [1, 3]), ['G10', 'G22']]

G10  G22
langs  level          
Python 1        0    0
       3        2    2
C      1        3    3
       3        5    5

## 分层索引的应用

对于拥有分层索引的数据集，可以指定层级`level`来进行数据聚合操作。例如，指定层级进行汇总统计：

In [76]:
newdf.sum(level='langs')

,G10,G14,G22
langs,,,
Python,3,3,3
C,12,12,12
